In [1]:

from osdatahub import FeaturesAPI, Extent, NGD
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os
import numpy as np
from convertbng.util import convert_bng, convert_lonlat

from folium.plugins import measure_control

key = os.environ.get('OS_API_KEY')
crs = "EPSG:27700"

In [2]:
def OSparam_feature(u,v,rad,product,key,clip):
    extent = Extent.from_radius((u,v), rad, "EPSG:27700")
    features = FeaturesAPI(key, product, extent)
    results = features.query(limit=500)
    if len(results['features']) == 0:
        out = 0
    TA_gdf = gpd.GeoDataFrame.from_features(results['features'])

    if clip == True:
        patch = Point(u,v).buffer(rad)
        d = {'col1': ['name1'], 'geometry': [patch]}
        patch_df = gpd.GeoDataFrame(d, crs="EPSG:27700")
        TA_gdf = TA_gdf.clip(patch)
    try:
        gd = (TA_gdf['Theme'] == 'Buildings') & (~np.isnan(TA_gdf['RelH2']))
        out = np.average(TA_gdf['RelH2'][gd], weights=TA_gdf['Shape_Area'][gd])
    except TypeError:
        out = 0
    return out, TA_gdf



In [3]:
X = 180700.
Y = 44777.
radius = 50
clip = True
product = 'topographic_area'

In [4]:
def building_height_radius(X, Y, radius,product,key, clip):
    area, TA = OSparam_feature(X, Y, radius, product,key, clip)
    TA = TA.set_crs(27700)
    TA_ll = TA.to_crs(4326)
    # lon, lat = convert_lonlat(X,Y)
    # map = TA_ll.explore('RelH2') ## colour by height
    # folium.Marker([lat[0], lon[0]],popup=[lat[0],lon[0]]).add_to(map)
    return TA_ll
    

In [5]:
BuildingData = building_height_radius(X, Y, radius,product,key, clip)


In [6]:
BuildingGeoSeries = BuildingData['geometry'][BuildingData['Theme'] == 'Buildings']
BuildingGeoSeries = BuildingGeoSeries.to_crs(27700)
BuildingDistances = BuildingGeoSeries.distance(Point(X,Y))
LineDistances = BuildingGeoSeries.shortest_line(Point(X,Y))

In [7]:
import folium
from shapely.geometry import Point, LineString

# Define the line and start/end points
line = LineDistances.iloc[0]
end_point = Point(line.coords[0])
start_point = Point(line.coords[-1])
line_points = [p for p in line.coords]
lats = convert_lonlat(*zip(*line_points))[0]
lons = convert_lonlat(*zip(*line_points))[1]

points = []
for i in range(len(lats)):
    points.append([lons[i], lats[i]])

start_lon, start_lat = convert_lonlat(start_point.x, start_point.y)
end_lon,end_lat = convert_lonlat(end_point.x, end_point.y)

# Define the projection
import pyproj
geodesic = pyproj.Geod(ellps='WGS84')
fwd_azimuth,back_azimuth,distance = geodesic.inv(start_lon[0], start_lat[0], end_lon[0], end_lat[0])

# Create a folium map centered on the line
m = folium.Map(location=[start_lat[0],start_lon[0]], zoom_start=19)

# # Add the line to the map in blue
folium.PolyLine(points, color='blue',popup=fwd_azimuth).add_to(m)

# # Add the start point to the map in red
folium.Marker(location=[start_lat[0],start_lon[0]], icon=folium.Icon(color='green')).add_to(m)

# # Add the end point to the map in green
folium.Marker(location=[end_lat[0], end_lon[0]], icon=folium.Icon(color='red')).add_to(m)

# # Display the map
m
# points